In [253]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

--2022-03-09 23:27:40--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
SSL_INIT
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg.6’

La_noche_estrellada 100%[===================>] 218.48K   336KB/s    in 0.7s    

2022-03-09 23:27:42 (336 KB/s) - ‘La_noche_estrellada1.jpg.6’ saved [223725/223725]



In [254]:
# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

--2022-03-09 23:27:42--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
SSL_INIT
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) [image/jpeg]
Saving to: ‘775px-Neckarfront_Tübingen_Mai_2017.jpg.5’

775px-Neckarfront_T 100%[===================>] 149.43K   285KB/s    in 0.5s    

2022-03-09 23:27:44 (285 KB/s) - ‘775px-Neckarfront_Tübingen_Mai_2017.jpg.5’ saved [153015/153015]



In [510]:
# Creamos el directorio para los archivos de salida
!mkdir ./content/output

mkdir: cannot create directory ‘./content/output’: File exists


In [511]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse
import tensorflow as tf
from keras.applications import vgg19
from keras import backend as K
from pathlib import Path


In [512]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida
#originales
#base_image_path = Path("./775px-Neckarfront_Tübingen_Mai_2017.jpg")
#style_reference_image_path = Path("./La_noche_estrellada1.jpg")


base_image_path = Path("./playa.jpg")
style_reference_image_path = Path("./wave_estilo.jpg")
result_prefix = Path("./content/output")

iterations = 100

In [513]:
# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

In [514]:
# originales
# total_variation_weight = 0.1
# style_weight = 10
# content_weight = 1

total_variation_weight = 0.2
style_weight = 100
content_weight = 1


# Respuesta:
# La funcion loss que esta minimizando el paper se compone de dos elementos:
#
# el elemento que representa el contenido de la imagen  (content_weight) y el elemento que representa el estilo de la imagen (style_weight)
# Entonces dependiendo de la proporcion de cada elemento la imagen resultante sera mejor o peor
# si se le da mas peso al contenido, la imagen tendra pocos rasgos del estilo de la segunda imagen
# en cambio si se da mas peso al estilo, se podria perder la estructura inicial de la primera imagen siendo irreconocible.
# total_variation_weight tiene la relacion entre content_weight/style_weight
# alfa (content_weight)/ beta (style_weight)
# representaria los valores de la siguiente ecuacion presentada en el paper.

In [515]:
#%% md
![ecuacionLoss.png](attachment:ecuacionLoss.png)

zsh:1: number expected


In [516]:
# hack to error: Could not import PIL.Image. The use of `load_img` requires PIL.
# uninstall pillow from conda and install by pip
# NO LO USE
#import sys
#from PIL import Image
#sys.modules['Image'] = Image 

#from PIL import Image
#print(Image.__file__)

#import Image
#print(Image.__file__)

In [517]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [518]:
width

775

In [519]:
height

596

In [520]:
img_ncols

520

In [521]:
# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?
#Ayuda: https://keras.io/applications/ => https://keras.io/api/applications/vgg/#vgg19-function

#Respuesta:
# cada keras application espera un tipo especifico de formato y tamano del vector
# por ejemplo vgg19 requiere que las imagenes esten en BGR
# y no en RGB. y centrara en zero cada canal de color con respecto al dataset ImageNet.
# img # (400, 517, 3)
# img after preprocess # (1, 400, 517, 3)
# de acuerdo a la documentacion de VGG19 es necesario expandir el array en 1 dimension para poder procesar en batch.

In [522]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [523]:

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

# Respuesta:
# como las imagenes para ser usadas en VGG19 necesitan un formato especifico BGR y la normalizacion centrada en 0.
# la salida necesita tener el  proceso inverso para obtener la imagen en el formato RGB, y los valores correctos en los canales.

In [524]:

def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [525]:
# style image
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [526]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

In [527]:
#Aclaración:

#La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [528]:

# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [529]:

# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


In [530]:
layer_features2 = outputs_dict['block1_conv1'] # 400,517,3 or 64
# layer_features2[1, :, :, :] 
# layer_features2[2, :, :, :] 
layer_features2

<tf.Tensor 'block1_conv1_8/Relu:0' shape=(3, 400, 520, 64) dtype=float32>

In [531]:
outputs_dict['block1_conv1'][2] # 400,517,3 or 64

<tf.Tensor 'strided_slice_274:0' shape=(400, 520, 64) dtype=float32>

In [532]:
outputs_dict['block2_conv1'][0]

<tf.Tensor 'strided_slice_275:0' shape=(200, 260, 128) dtype=float32>

In [533]:
outputs_dict['block3_conv1'][2]

<tf.Tensor 'strided_slice_276:0' shape=(100, 130, 256) dtype=float32>

In [534]:
outputs_dict['block4_conv1'][2]

<tf.Tensor 'strided_slice_277:0' shape=(50, 65, 512) dtype=float32>

In [535]:
outputs_dict['block5_conv1'][2]

<tf.Tensor 'strided_slice_278:0' shape=(25, 32, 512) dtype=float32>

In [536]:
# 4) En la siguientes celdas:

#- ¿Qué es la matriz de Gram?¿Para qué se usa?

#Es una matriz que contiene el producto punto entre dos vectores para extraer el estilo (textura) de una imagen, esto indica cuan similar son estos dos vectores, para luego
#calcular la funcion de optimizacion "style loss". Este producto punto devuelve informacion sobre la textura de la imagen y
#Zero informacion sobre su estructura espacial (Contenido).

#Gram matrix es el resultado del producto punto de un vector que representa "features maps" de una cnn.
#un feature map puede ser textures, brush strokes, lines, curves, dots, color distribution.
#Esto usa vectores aplanados de tamano C a partir de la capa de una cnn de profundidad C. (CxC)
# Para cada layer se genera una gram matrix entonces de esto se calcula el mse loss, y sumariza por cada capa.

# En sintesis Gram matrix se usa para capturar el estilo a partir de caracteristicas "features map / features metrics" de una CNN.

#- ¿Por qué se permutan las dimensiones de x?
# Para tener una estructura donde se pueda realizar el producto punto, requerido por la matrix transpuesta.
# x consiste en un tensor donde la posicion 2 corresponde a un batch de 64
# la posicion 0 e 1 corresponde al numero de filas y columnas que representan a la imagen.

In [537]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [538]:

# 5) Losses:

#Explicar qué mide cada una de las losses en las siguientes tres celdas.

#Rta: 
# style_loss mide que tan cercano es el estilo de la imagen de referencia de estilo con la imagen de entrada base.
# utilizando este valor permite obtener la textura o estilo de una imagen y
# minimizando esta diferencia a traves de las distintas capas de la red.
#
# content_loss 
# contiene la distancia o medicion que tan cercana es la imagen de entrada base con la imagen combinada (la imagen nueva con parte del estilo de referencia)
# donde si bien mientras mas cercana al contenido mas puede estar alejada del estilo de referencia
# por lo que es necesario la combinacion de ambas metricas tanto el style loss como el content loss 
# para obtener una imagen coherente y no perder el 100%  de la  estructura de la imagen base.
# 
# total_variation_loss
# total_variation_loss(combination_image)
# es como un suavisado de la imagen, toma la matriz o tensor que representa la imagen nueva generada 
# y reduce que se produzcan fuertes cambios en los valores. 
# y evitando la eliminacion ruido blanco de la imagen nueva generada.

In [539]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [540]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [541]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [542]:

# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [543]:

# #initialize the variable
# #init_op = tf.initialize_all_variables()
# init = tf.compat.v1.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init_op) #execute init_op
#     #print the random values that we sample
#     print (sess.run(loss))

In [544]:

#import tensorflow as tf
#tf.compat.v1.disable_eager_execution()

#from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()


grads = K.gradients(loss, combination_image)
#grads = tf.GradientTape(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

In [545]:

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

#Respuesta:

# fmin_l_bfgs_b 
# es un algoritmo  de la familia de metodos Quasi-Newton, donde tiene como objetivo optimizar una funcion y buscar 
# los extremos locales de una funcion. Trabaja bien con datasets largos ya que necesita menos memoria que el algoritmo estandar BFGS.
# En este caso se utiliza para minimizar *la funcion global loss* que se compone  de los siguientes elementos: style loss, content loss, y total loss.

# ¿En qué se diferencia con la implementación del paper? 
# En el paper usan el descenso del gradiente para minimizar las loss. 
# Y no se visualiza el uso de alguna funcion o Ecuacion de suavizado sobre la imagen nueva generada como 
# "total_variation_loss" es usada en esta notebook para ese suavizado.
# ¿Se puede utilizar alguna alternativa? Si, standard error back-propagation. o descenso del gradiente

# celda 1 = eval_loss_and_grads funcion que 
# evalua la loss y el valor del gradiente para la imagen X, x seria la imagen nueva generada.
# devuelve el valor de la loss y el gradiente en el tipo de dato float64
#
# celda 2 = evaluator
# es una clase que representa al Evaluador y tiene dos atributos loss y los valores del gradiente
# inicializa los valores en Null, y usa la funcion de la celda anterior (eval_loss_and_grads) 
# para ir variando los valores de la loss
# y el valor del gradiente

# celda 3 = implentacion del uso de evaluator
# preprocesa la imagen  (normalizacion y pasar de BGR a RGB)
# de acuerdo a la cantidad de iteraciones definidas para cada iteracion:
# utiliza el algoritmo "" para minimizar la loss y lo guarda en una variable ""
# luego copia el vector (x.copy()), despreprocesa la imagen (BGR -> RGB) y guarda la imagen para esa iteracion.


In [546]:

def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.


In [547]:

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [548]:
# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:

evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 108933950000.0
Image saved as content/output/output_at_iteration_0.png
Iteration 0 completed in 357s
Start of iteration 1
Current loss value: 57507586000.0
Image saved as content/output/output_at_iteration_1.png
Iteration 1 completed in 381s
Start of iteration 2
Current loss value: 41749480000.0
Image saved as content/output/output_at_iteration_2.png
Iteration 2 completed in 364s
Start of iteration 3
Current loss value: 33484022000.0
Image saved as content/output/output_at_iteration_3.png
Iteration 3 completed in 365s
Start of iteration 4
Current loss value: 28370323000.0
Image saved as content/output/output_at_iteration_4.png
Iteration 4 completed in 366s
Start of iteration 5
Current loss value: 24218776000.0
Image saved as content/output/output_at_iteration_5.png
Iteration 5 completed in 366s
Start of iteration 6
Current loss value: 21334706000.0
Image saved as content/output/output_at_iteration_6.png
Iteration 6 completed in 371s
Start of ite

Current loss value: 2990881800.0
Image saved as content/output/output_at_iteration_58.png
Iteration 58 completed in 366s
Start of iteration 59
Current loss value: 2948320500.0
Image saved as content/output/output_at_iteration_59.png
Iteration 59 completed in 371s
Start of iteration 60
Current loss value: 2887857700.0
Image saved as content/output/output_at_iteration_60.png
Iteration 60 completed in 372s
Start of iteration 61
Current loss value: 2812857300.0
Image saved as content/output/output_at_iteration_61.png
Iteration 61 completed in 371s
Start of iteration 62
Current loss value: 2757162000.0
Image saved as content/output/output_at_iteration_62.png
Iteration 62 completed in 367s
Start of iteration 63
Current loss value: 2699862800.0
Image saved as content/output/output_at_iteration_63.png
Iteration 63 completed in 369s
Start of iteration 64
Current loss value: 2657243400.0
Image saved as content/output/output_at_iteration_64.png
Iteration 64 completed in 370s
Start of iteration 65

In [ ]:

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

#Respuesta:




In [ ]:
# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

#Respuesta:
# combinaciones de pesos de las losses
# primera ejecucion:
# total_variation_weight = 0.1
# style_weight = 10
# content_weight = 1

In [ ]:
# combinaciones de pesos de las losses
# segunda ejecucion:
# total_variation_weight = 0.2
# style_weight = 100
# content_weight = 1
# alfa (content_weight)/ beta (style_weight)
# cambio un poco la intensidad de los colores, con esta combinacion tiene colores mas oscuros y mate
# tiene pocas diferencias con respecto a la anterior configuracion pero se nota un poco en los colores 
# tendiendo hacia la paleta de colores de la imagen de estilo de referencia.

In [ ]:
# combinaciones de pesos de las losses
# tercera ejecucion:
# total_variation_weight = 0.1
# style_weight = 1
# content_weight = 10

#se visualiza que las lineas, formas y estructura del contenido se mantiene,  muy poco varia a diferencia de la anterior 
# ejecucion donde las luces, y formas circulares tomaban mas la estructura de la imagen de referencia del estilo.

In [ ]:
# combinaciones de pesos de las losses
# cuarta ejecucion:
# total_variation_weight = 0.1
# style_weight = 1
# content_weight = 10000
# muy claro que muestra pocos cambios de estilo y la paleta de colores es identica a la fotografia original
# conserva claramente la estructura del contenido y tambien los colores, algunas lineas tienen el estilo de
# la fotografia de referencia del estilo.

In [ ]:
# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

#Respuesta:

# quinta ejecucion
# imagenes diferentes
# pesos 
# total_variation_weight = 0.1
# style_weight = 1
# content_weight = 10000
# no se vio ningun cambio.

In [ ]:
# sexta ejecucion
# total_variation_weight = 0.2
# style_weight = 100
# content_weight = 1